## Bag-of-Words representation

1. 크롤링
2. Collection => doc 집합
3. BOW => Word (Tokenize, Preprocessing)
4. Lexicon Dictionary (|V|) => 전체 문서의 token 리스트
5. 문서표현 (by BOW) => Lexicon Dictionary의 token이 해당 문서에 있으면 1, 없으면 0으로 표현된 리스트

## Lecixon List

In [2]:
from konlpy.corpus import kobill

# lexicon을 만드는 함수 정의
def getLexicon():
    lexicon = list() # unique terms list
    # 사전(lexicon) 만들기 -> 우리가 수집한 데이터(crawled data)를 N차원으로 표현
    for docName in kobill.fileids():
        document = kobill.open(docName).read()

        for token in document.split():
            if token not in lexicon: # token이 lexicon에 있는지 검사 후 있으면 추가를 하지 않고 없으면 추가를 한다.
                lexicon.append(token)
                
    return lexicon

In [3]:
# 수행시간 비교를 위한 timeit 출력 
%timeit getLexicon()

82.9 ms ± 2.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Lecixon List -> Set

In [5]:
# set을 이용하는 것으로 getLexicon 함수 수정 (마이크로세컨드에서 나노세컨드로 매우 빨라짐)
def getLexiconBySet(): 
    lexicon = list()
    
    for docName in kobill.fileids():
        document = kobill.open(docName).read()

        for token in document.split():
            lexicon.append(token)
                
    return list(set(lexicon)) # set을 함으로서 중복된 값들이 사라진다.

In [6]:
# 수행시간 비교를 위한 timeit 출력
%timeit getLexiconBySet()

4.07 ms ± 87.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


* 단어사전 [0:단어1, 1:단어2, 2:단어3, ...]
* 문서 0: [0:0 or 1] => 문서에 있으면 1, 없으면 0으로 표현
* 1:
* 문서목록 [0:문서1, 1:문서2, ...]
* 문서: [0, 1, 0, 1]

In [7]:
lexicon = getLexiconBySet()

In [9]:
def getDocRepr(lexicon):
    # 문서를 전체 lexicon이라는 사전으로 표현한 후 사전의 전체 개수(유니크한 단어의 수)를 N개라고 할 때 N차원으로 문서를 하나 표현하고자 함. 
    # 단어가 있으면 1, 없으면 0으로 표현한 후 append. 따라서 사전에 위치한 단어의 인덱스를 받아올 필요가 있으며, 그 결과 1개의 문서에 대한 벡터 표현이 가능해진다. 
    
    docList = list() # 문서목록
    docRepr = list() # 문서표현 of BOW의 집합 => 문서갯수만큼
    
    for docName in kobill.fileids(): # 문서 한개 열고
        document = kobill.open(docName).read()
        
        docList.append(docName) # 문서목록에 추가
        docVector = list(0 for _ in range(len(lexicon))) # lexicon의 사이즈만큼 0부터 돌아가면서 0으로 초기화한다. 즉 2300차원 정도를 갖는 0으로 초기화된 벡터가 됨. 
        # 문서표현(BOW) = [0] * 단어의 갯수(|Lexicon|)
        
        for token in document.split(): # 문서내 단어
            if token in lexicon: # 사전에 있는지
                docVector[lexicon.index(token)] = 1 # 사전에 있으면, 사전의 단어 위치(index)에 1을 넣어줌.(몇번째에 위치하는지 인덱스를 받고 1을 넣어줌. 
        docRepr.append(docVector) # 문서 Vector를 docRepr에 Append (docVector는 벡터화된 lexicon, docRepr는 그런 docRepr들의 집합)
    return docList, docRepr

In [10]:
# 수행시간 비교를 위한 timeit 출력
%timeit getDocRepr(lexicon)

359 ms ± 14.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
def getDocReprByDict(lexicon): # defaultdict로 속도를 개선한 getDocRepr 함수
    # docRepr = defaultdict(lambda: defaultdict(int)) # 람다함수로 초기화시키는 방법
    docRepr = dict() 
    # key = 문서
    # value = BOW => list X, dict
    
    for docName in kobill.fileids():
        document = kobill.open(docName).read()
        
        docRepr[docName] = dict()
        
        for token in document.split(): 
            if token in lexicon: 
                docRepr[docName][lexicon.index(token)] = 1 # 사전에 있으면, 사전의 단어 위치(index)에 1을 넣어줌.(몇번째에 위치하는지 인덱스를 받고 1을 넣어줌. 
    
    return docRepr

# --> bag of word
# Document-Term Matrix라고 하여 DTM으로 부른다. 
#     w1, w2, w3, ..., w2300
# d1   1,  0,  0, ..., => sparse하다
# d2
# d3
# ...
# d10
# dict{dict{}} 구조로 되어 있는 형태. 

In [21]:
%timeit getDocReprByDict(lexicon)

337 ms ± 8.43 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
docRepr = getDocReprByDict(lexicon)

In [23]:
len(docRepr), list(docRepr.keys())[0]

(10, '1809890.txt')

In [27]:
docRepr['1809890.txt'][0], lexicon[0]

(1, '공무원의')

In [28]:
from collections import defaultdict

def getDocReprByDefaultDict(lexicon):
    docRepr = defaultdict(lambda: defaultdict(int))
    
    for docName in kobill.fileids():
        document = kobill.open(docName).read()
 
        for token in document.split(): 
            docRepr[docName][token] = 1
    
    return docRepr

In [29]:
%timeit getDocReprByDefaultDict(lexicon)

3.66 ms ± 36.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
docRepr = getDocReprByDefaultDict(lexicon)

In [31]:
docRepr['1809890.txt']

defaultdict(int,
            {'지방공무원법': 1,
             '일부개정법률안': 1,
             '(정의화의원': 1,
             '대표발의': 1,
             ')': 1,
             '의': 1,
             '안': 1,
             '번': 1,
             '호': 1,
             '9890': 1,
             '발의연월일': 1,
             ':': 1,
             '2010.': 1,
             '11.': 1,
             '12.': 1,
             '발': 1,
             '자': 1,
             '정의화․이명수․김을동': 1,
             '이사철․여상규․안규백': 1,
             '황영철․박영아․김정훈': 1,
             '김학송': 1,
             '의원(10인)': 1,
             '제안이유': 1,
             '및': 1,
             '주요내용': 1,
             '초등학교': 1,
             '저학년의': 1,
             '경우에도': 1,
             '부모의': 1,
             '따뜻한': 1,
             '사랑과': 1,
             '보살핌이': 1,
             '필요': 1,
             '한': 1,
             '나이이나,': 1,
             '현재': 1,
             '공무원이': 1,
             '자녀를': 1,
             '양육하기': 1,
             '위하여': 1,
             '육아휴직을': 1,
      

~~~
DTM => TDM 변환
검색 => Input(키워드들의 집합 = Query) 
키워드들은 단어로 구성, 단어를 추출

Q = ['국회', '의원'] 이라면,

|Q| = q, |D| = d, |V| = v 라면,
qd, v ==> 너무 과다한 계산 (실행 불가)

for 단어 in Q:
for 전체 문서를 대상:
for 문서 내 단어 :
if 단어 == 사전:
이때의 해당 문서가 검색 후보

이를 개선하기 위해,
InvertedDocument (역문헌 구조)를 사용
key: 단어 in 사전,
value: 어느 문서에서 나왔는지, 몇 번(, 어느 위치 등)

for 단어 in Q:

    # for 전체 문서를 대상:  # 전체 문서를 대상으로 찾을 필요 없음
    #     for 문서 내 단어 :
    #         if 단어 == 사전:
    단어 in InvertedDocument.keys():
                이때의 해당 문서가 검색 후보

    DTM(Document-Term Matrix)
    w1, w1, w3, ... wn
d1 1, 0, 1, 0, ... , 0 => Sparse
d2
d3

Query 
    W1, W1, W3, ... , 1
q1 
q2
q3
DTM => TDM으로 변경 (리스트 x, 딕셔너리 o => 공간을 줄일려고)

    TDM(Term-Document Matrix)
    d1, d2, d3, ... , d10
t1 1, 0, 1, 0, 1, 0, 0, 0 => Sparse => Dict
t2
t3
...
tn
~~~

~~~
Inverted document(역문헌구조)를 통해 효율성을 높이고자 함. 

Document-Term Matrix(DTM) 
    w1, w2, w3, ..., w2300
d1   1,  0,  0, ..., => sparse하다
d2
d3
...
d10

Term-Document Matrix(TDM) -> 단순히 축만 바뀐 형태.  
    d1, d2, d3, ..., w10
w1   1,  0,  0, ..., => sparse=>dict
w2
w3
...
w2300
~~~

In [49]:
# invertedDocument (역문헌구조, 어휘)

# 이 구조는 파이썬에서만 사용 가능. Dictionary | Postiong DB로 구성
# 다른 언어에서 짜려면(eg. C) 다음과 같이 코딩 :
    # Dictionary => term, fp (메모리에 Hash 형태로 관리되므로 매우 빠름)
    # Postiong => struct(docid, freq, next=fp) (fileDB)
    
# boolean 기반의 모델은 0이 너무 많은 sparse한 경우에 불리하다. 따라서 아래와 같은 invertedDocument 방식이 더 나음. 
def invertedDocument(DTM): # Transpose를 하지 않는 이유는 우리가 다루고자 하는 데이터가 몇십, 몇백만이기 때문에 Numpy array를 활용할 수 없기 때문. 
    TDM = defaultdict(lambda: defaultdict(int))
    
    # Only python => Dictionary | Posting DB
    # Dictionary => term, fp (Hash in memory)
    # Posing => struct(docid, freq, next=fp) (FileDB)
    
    for docName, docVector in DTM.items():  
        for term, freq in docVector.items():
            TDM[term][docName] = freq
            
    return TDM

In [50]:
TDM = invertedDocument(docRepr)

In [51]:
TDM["국회"], TDM["의원"] # 어느 문서에 해당 단어가 들어 있는지 검색

(defaultdict(int, {'1809897.txt': 1, '1809898.txt': 1}),
 defaultdict(int, {'1809896.txt': 1}))

In [52]:
TDM["국회"] or TDM["의원"]

defaultdict(int, {'1809897.txt': 1, '1809898.txt': 1})